### Import Libraries

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files
import csv # library to handle csv files

import geocoder as geocoder
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

from bs4 import BeautifulSoup
import requests

print('Libraries imported.')

Libraries imported.


### 1. London neighbbourhood informatio - Scraping data from Wikipedia

In [4]:
dfs = pd.read_html('https://en.wikipedia.org/wiki/List_of_areas_of_London')
raw = dfs[1]
raw.head()

df1 = raw.drop(raw.index[0])
df1.columns = ['Location','London_borough','Post town','Postcode district','Dial Code','OS grid ref']
df1.head()

,Location,London_borough,Post town,Postcode district,Dial Code,OS grid ref
1,Abbey Wood,"Bexley, Greenwich [7]",LONDON,SE2,020,TQ465785
2,Acton,"Ealing, Hammersmith and Fulham[8]",LONDON,"W3, W4",020,TQ205805
3,Addington,Croydon[8],CROYDON,CR0,020,TQ375645
4,Addiscombe,Croydon[8],CROYDON,CR0,020,TQ345665
5,Albany Park,Bexley,"BEXLEY, SIDCUP","DA5, DA14",020,TQ478728


In [5]:
# Drop the unnecessary information and clean data
df = df1.drop(['Dial Code','OS grid ref'], axis = 1)
df = df.reindex()
df.head()

,Location,London_borough,Post town,Postcode district
1,Abbey Wood,"Bexley, Greenwich [7]",LONDON,SE2
2,Acton,"Ealing, Hammersmith and Fulham[8]",LONDON,"W3, W4"
3,Addington,Croydon[8],CROYDON,CR0
4,Addiscombe,Croydon[8],CROYDON,CR0
5,Albany Park,Bexley,"BEXLEY, SIDCUP","DA5, DA14"


In [6]:
df.shape

(533, 4)

get logitude and altitude of London Neighbourhood

In [7]:
def getLatLong(row):
    #print('post :{}'.format(row[:]))
    #print('neigh :{}'.format(row[1]))
    # initialize your variable to None
    lat_lng_coords = None
    search_query = '{}, London,UK'.format(row)
    # loop until you get the coordinates
    try:
        while(lat_lng_coords is None):
            #g = geocoder.here(search_query,app_id=app_id,app_code=app_code)
            g = geocoder.arcgis(search_query)
            lat_lng_coords = g.latlng
            #print('FIRST')
    except IndexError:
        latitude = 0.0
        longitude = 0.0
        print('BACKUP')
        return [latitude,longitude]

    latitude = lat_lng_coords[0]
    longitude = lat_lng_coords[1]
    print(latitude, longitude)
    return [latitude, longitude]

In [8]:
# coords_list = df['Postcode district'].apply(getLatLong).tolist()

51.492450000000076 0.12127000000003818
51.51324000000005 -0.2674599999999714
51.38475500000004 -0.051498623999975734
51.38475500000004 -0.051498623999975734
51.452067760578885 0.1722299153689439
51.50642000000005 -0.1272099999999341
51.51200000000006 -0.08057999999994081
51.51651000000004 -0.11967999999995982
51.50642000000005 -0.1272099999999341
51.41009000000008 -0.05682999999993399
51.523610000000076 -0.09876999999994496
51.27172802817503 0.07471050704375404
51.56393000000003 -0.12944999999996298
51.58370000000008 0.17476000000004888
51.615680000000054 -0.2451099999999542
51.63429000000008 -0.13365999999996347
51.44822000000005 -0.1483899999999494
51.499600000000044 -0.09612999999995964
51.523610000000076 -0.09876999999994496
51.53312000000005 0.08407653200004006
51.50642000000005 -0.1272099999999341
51.47052000000008 0.14670637700004363
51.47457000000003 -0.24211999999994305
51.452067760578885 0.1722299153689439
51.615680000000054 -0.2451099999999542
51.64441500000004 -0.1791854279

51.589270000000056 -0.16394999999994297
51.50642000000005 -0.1272099999999341
51.50642000000005 -0.1272099999999341
51.35142941071637 -0.24988108535584375
51.50878000000006 -0.3362999999999374
51.44864734579377 -0.3294047593453292
51.50642000000005 -0.1272099999999341
51.53916000000004 -0.2512299999999641
51.50510549999999 -0.4275742499999865
51.50642000000005 -0.1272099999999341
51.50642000000005 -0.1272099999999341
51.50642000000005 -0.1272099999999341
51.50642000000005 -0.1272099999999341
51.58151000000004 -0.07589999999993324
51.567890615208455 -0.3394832478241024
51.567890615208455 -0.3394832478241024
51.50642000000005 -0.1272099999999341
51.61291065511444 -0.37696639802957227
51.448444494776844 -0.44849227524521545
51.66208623796546 0.18662121631421688
51.50642000000005 -0.1272099999999341
51.50510549999999 -0.4275742499999865
51.50642000000005 -0.1272099999999341
51.58912000000004 -0.2280999999999267
51.45529000000005 -0.09927999999996473
51.50642000000005 -0.1272099999999341
51

51.50642000000005 -0.1272099999999341
51.50642000000005 -0.1272099999999341
51.45066000000003 -0.18996999999995978
51.63429000000008 -0.13365999999996347
51.52022000000005 -0.054309999999929914
51.392201353000075 -0.2159499999999639
51.49713000000003 -0.13828999999992675
51.45374249766627 -0.32996134872866295
51.51651000000004 -0.11967999999995982
51.462680000000034 -0.035579999999924894
51.53398000000004 -0.17377999999996518
51.523610000000076 -0.09876999999994496
51.50642000000005 -0.1272099999999341
51.524500000000046 -0.12272999999993317
51.50642000000005 -0.1272099999999341
51.58151000000004 -0.07589999999993324
51.62043771181625 -0.30130161650491155
51.52022000000005 -0.054309999999929914
51.476230000000044 -0.13055999999994583
51.56350000000003 -0.07312999999993508
51.53916000000004 -0.2512299999999641
51.540140000000065 0.0027800000000297587
51.45374249766627 -0.32996134872866295
51.41870000000006 -0.12752999999997883
51.56878000000006 -0.09846999999996342
51.567890615208455 -0

### Merge to the previous dataframe

In [9]:
df[['Latitude','Longitude']]=pd.DataFrame(coords_list,columns=['Latitude', 'Longitude'])
df.head()

,Location,London_borough,Post town,Postcode district,Latitude,Longitude
1,Abbey Wood,"Bexley, Greenwich [7]",LONDON,SE2,51.513240,-0.267460
2,Acton,"Ealing, Hammersmith and Fulham[8]",LONDON,"W3, W4",51.384755,-0.051499
3,Addington,Croydon[8],CROYDON,CR0,51.384755,-0.051499
4,Addiscombe,Croydon[8],CROYDON,CR0,51.452068,0.172230
5,Albany Park,Bexley,"BEXLEY, SIDCUP","DA5, DA14",51.506420,-0.127210


In [10]:
# Save it to a CSV file
df.to_csv(r'./LondonPostCodeDistrict.csv')

### 2. Must-visit places information - from one (or multiple) review article https://foursquare.com/gmissawa/list/places-to-visit-in-london

In [13]:
#Get the data with beautifulsoup
source = requests.get('https://foursquare.com/gmissawa/list/places-to-visit-in-london').text
soup = BeautifulSoup(source,'lxml')
#print(soup.prettify())

In [14]:
# Scraping the venue information
match = soup.find('div', class_='venueInfo')
#print(match.prettify())

In [197]:
# Scraping trial, to find all the information in the first entry
# venue_name = match.find('p', class_ = 'venueName')
# rating = match.find('div', class_ = 'rating')
# category = match.find('span', class_ = "unlinkedCategory")

# print (venue_name.text)
# print(rating.text)
# print(category.text)

In [211]:
# Save as csv
csv_file = open('London_venue.csv','w')
csv_writer = csv.writer(csv_file, lineterminator='\n')
csv_writer.writerow(['Venue Name', 'Category', 'Rating', 'Venue ID'])

# Loop through all the venues and save the inforamtion into a list
for match in soup.find_all('div', class_='venueInfo'):
    
    venue_name = match.find('p', class_ = 'venueName').text
    category = match.find('span', class_ = "unlinkedCategory").text
#     print (venue_name)
#     print(category)
    
    try:
        rating = match.find('div', class_ = 'rating').text
#         print(rating)
        venue_src = match.find('a')['href']
        venue_id = venue_src.split('/')[3]
#         print(venue_id)
    except Exception as e:
        rating = 'NaN'
#         print('NaN')
        veneue_id = 'NaN'
    # Save as csv
    csv_writer.writerow([venue_name, category, rating, venue_id])

csv_file.close()

In [27]:
# Read csv file as pandas dataframe
df_ln = pd.read_csv("London_venue3.csv", encoding = "ISO-8859-1")
# print(df_ln.head())
# print(df_ln.shape)

df_2 = df_ln.rename(columns={'Venue Name':'name', 'Category':'category','Rating':'rating','Venue ID':'ID'})
df_2.head(20)

#df.rename(columns={"A": "a", "B": "b", "C": "c"}, errors="raise")
#df_2 = pd.DataFrame(df_ln, columns = ['name', 'category', 'rating','ID'])
#df_2.head()


,name,category,rating,ID
0,Big Ben (Elizabeth Tower),Monument / Landmark,9.3,4ac518cef964a520f6a520e3
1,The London Eye,Scenic Lookout,9.2,4ac518cef964a52021a620e3
2,Trafalgar Square,Plaza,9.2,4ac518cef964a520f9a520e3
3,Tower of London,Castle,9.2,4ac518cef964a520f7a520e3
4,Westminster Abbey,Church,9.2,4ac518cdf964a520eea520e3
5,British Museum,History Museum,9.4,4ac518d2f964a5203da720e3
6,St Paul's Cathedral,Church,9.2,4ac518cef964a520f5a520e3
7,Tower Bridge,Bridge,9.5,4ac518cef964a520f8a520e3
8,Buckingham Palace,Palace,8.7,4abe4502f964a520558c20e3
9,Piccadilly Circus,Plaza,9.0,4c9c6a059c48236a1cb14dee


#### Get more informaiton from Foursquare based on venue ID (This part can be skipped, since the inforamtion are saved into a .csv file)

The webscraping step has got some venue id that corresponse to the same venue, the venue informaiton of 78 is received. Due to the number of requests for Foursquare is limited, in this work, we will use 78 popular venunes' information as an case study. This part can be improved in the future to include all the poluplar venues.

In [29]:
# Directly read from the csv file, the code used to generate the csv file is in Appendix Section
df_venue = pd.read_csv("df_venue.csv")
#print(df_venue.head())
print(df_venue.shape)
df_venue.head(20)

(78, 7)


,Unnamed: 0,location.lat,location.lng,location.postalCode,name,rating,likes.count
0,37,51.512726,-0.201859,W11 2QD,Alice's,8.2,9
1,19,51.512841,-0.127896,WC2H 9ND,Ambassadors Theatre,8.1,73
2,12,51.512211,-0.123566,WC2E 8HB,Apple Covent Garden,9.0,2180
3,32,51.519800,-0.093969,EC2Y 8DS,Barbican Art Gallery,9.2,273
4,36,51.519855,-0.093942,EC2Y 8DS,Barbican Centre,9.3,1763
5,5,51.519009,-0.126437,WC1B 3DG,British Museum,9.4,8498
6,8,51.500956,-0.143123,SW1A 1AA,Buckingham Palace,8.7,6627
7,20,51.541475,-0.146084,NW1 8AF,Camden Lock Market,9.1,3789
8,34,51.502079,-0.129305,SW1A 2AQ,Churchill War Rooms (Churchill Museum & Cabine...,9.1,503
9,10,51.511977,-0.122799,WC2E 8RF,Covent Garden Market,9.3,3409


### This is the second part of data of popular attractions in London with the latitude and logitude, number of likes, name of the venue and postcode.

In [14]:
#df_venue

In [28]:
# Merge df_venue with df_2
df = pd.merge(df_venue, df_2, on='name')
df.head()



,Unnamed: 0,location.lat,location.lng,location.postalCode,name,rating_x,likes.count,category,rating_y,ID


### Appendix: Code used to do webscraping and making foursquare API requests

#### Read the information from csv file (generated from Foursquare requests)

In [35]:
CLIENT_ID= 'EL0QYZH3NAVAKDR2M5EGMSKEFEM3BXPO2CHPEMNR4GDAF3VL'
CLIENT_SECRET= 'MYTF243JWF3F1RLYHNLQQXNQHBHMYFKEJJWAG0QKN0TTJRRS'

In [36]:
CLIENT_ID = '' # your Foursquare ID
CLIENT_SECRET = '' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 
CLIENT_SECRET:


In [ ]:
# # Loop through df_ln datafrmae limited to first 45 items (each account has 50 request/day)
# df_4s = pd.DataFrame([])
# for i in range(136):
#     venue_id = df_ln['Venue ID'][i]
#     #print(venue_id)
    
#     url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
#     #print(url)
    
#     #Request from foursqure
#     result = requests.get(url).json()
#     #print(result)
    
#     #assign relevant part of JSON to venue
#     venues = result['response']['venue']
#     dataframe = json_normalize(venues)
    
#     df_4s = df_4s.append(dataframe)

# #print(df_4s.head())

In [142]:
# data1 = pd.read_csv("place1to39.csv")
# data2 = pd.read_csv("place40to89.csv")
# data3 = pd.read_csv("place90to136.csv")
# columnsnew =['location.lat','location.lng', 'location.postalCode', 'name', 'rating', 'likes.count']
# data1_=data1[columnsnew]
# data2_=data2[columnsnew]
# data3_=data3[columnsnew]
# print('size of data1, data2, data3:', data1_.shape, data2_.shape, data3_.shape)

# dfall = data1_.append(data2_)
# dfall = dfall.append(data3_)

# print('size of dfall:', dfall.shape)

# #Remove duplicates
# dfall.sort_values('name', inplace=True)
# dfall.drop_duplicates(subset ="name", 
#                      keep = False, inplace = True) 
# dfall.shape
# # Save to a csv 
# dfall.to_csv(r'./df_venue.csv')

In [ ]:
# # Save the requests into csv file 
# df_4s.to_csv(r'./place45.csv')